In [1]:
# import packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import pickle
import gzip
import json

In [2]:
with open('../input/ctr-read-data/merged_df.pkl', 'rb') as inputfile:
    merged_df = pickle.load(inputfile)

In [3]:
train_df = merged_df.loc[:40000000, :]
test_df = merged_df.loc[40000001:, :]

In [4]:
def calc_smooth_mean(df, col, label, mean_label, weight):
    # Compute the number of values and the mean of each group
    agg = df.groupby(col)[label].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + weight * mean_label) / (counts + weight)

    mapping_dict = {key:value for key,value in zip(smooth.index.tolist(),smooth.values.tolist())}
    # Replace each value by the according smoothed mean and return mapping dict
    return df[col].map(smooth), mapping_dict

In [5]:
# Compute the global mean
mean_label = train_df["label"].mean()
list_mapping_dict = []

for col in train_df.columns.tolist():
    encoded, mapping_dict = calc_smooth_mean(train_df, col, "label", mean_label, 300)
    downcasted = pd.to_numeric(encoded , downcast='float')
    train_df[col+'_tenc'] = downcasted
    list_mapping_dict.append(mapping_dict)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
for col in train_df.columns:
    print(col, train_df[col].isnull().sum())

label 0
uid 0
task_id 0
adv_id 0
creat_type_cd 0
adv_prim_id 0
dev_id 0
inter_type_cd 0
slot_id 0
spread_app_id 0
tags 0
app_first_class 0
app_second_class 0
age 0
city 0
city_rank 0
device_name 0
device_size 0
career 0
gender 0
net_type 0
residence 0
his_app_size 0
his_on_shelf_time 0
app_score 0
emui_dev 0
list_time 0
device_price 0
up_life_duration 0
up_membership_grade 0
membership_life_duration 0
consume_purchase 0
communication_onlinerate 0
communication_avgonline_30d 0
indu_name 0
pt_d 0
label_tenc 0
uid_tenc 0
task_id_tenc 0
adv_id_tenc 0
creat_type_cd_tenc 0
adv_prim_id_tenc 0
dev_id_tenc 0
inter_type_cd_tenc 0
slot_id_tenc 0
spread_app_id_tenc 0
tags_tenc 0
app_first_class_tenc 0
app_second_class_tenc 0
age_tenc 0
city_tenc 0
city_rank_tenc 0
device_name_tenc 0
device_size_tenc 0
career_tenc 0
gender_tenc 0
net_type_tenc 0
residence_tenc 0
his_app_size_tenc 0
his_on_shelf_time_tenc 0
app_score_tenc 0
emui_dev_tenc 0
list_time_tenc 0
device_price_tenc 0
up_life_duration_tenc 0

In [7]:
for i, col in enumerate(test_df.columns.tolist()):
    encoded=test_df[col].map(list_mapping_dict[i])
    downcasted = pd.to_numeric(encoded , downcast='float')
    test_df[col+'_tenc']=downcasted

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
for col in test_df.columns:
    print(col, test_df[col].isnull().sum(), test_df[col].nunique())

label 0 2
uid 0 575695
task_id 0 3829
adv_id 0 4533
creat_type_cd 0 8
adv_prim_id 0 101
dev_id 0 55
inter_type_cd 0 4
slot_id 0 12
spread_app_id 0 69
tags 0 30
app_first_class 0 3
app_second_class 0 19
age 0 8
city 0 344
city_rank 0 4
device_name 0 88
device_size 0 161
career 0 9
gender 0 3
net_type 0 5
residence 0 36
his_app_size 0 21
his_on_shelf_time 0 4
app_score 0 2
emui_dev 0 16
list_time 0 17
device_price 0 7
up_life_duration 0 21
up_membership_grade 0 4
membership_life_duration 0 17
consume_purchase 0 9
communication_onlinerate 0 1377
communication_avgonline_30d 0 14
indu_name 0 38
pt_d 0 7
label_tenc 0 2
uid_tenc 1839 6288
task_id_tenc 3 3465
adv_id_tenc 3 3775
creat_type_cd_tenc 0 8
adv_prim_id_tenc 0 101
dev_id_tenc 0 55
inter_type_cd_tenc 0 4
slot_id_tenc 0 12
spread_app_id_tenc 0 69
tags_tenc 0 30
app_first_class_tenc 0 3
app_second_class_tenc 0 19
age_tenc 0 8
city_tenc 0 344
city_rank_tenc 0 4
device_name_tenc 0 88
device_size_tenc 1 156
career_tenc 0 9
gender_tenc 0 3
n

In [9]:
test_df.to_csv("test_df.csv")

In [10]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1907132 entries, 40000001 to 41907132
Data columns (total 72 columns):
 #   Column                            Dtype   
---  ------                            -----   
 0   label                             int8    
 1   uid                               int32   
 2   task_id                           int16   
 3   adv_id                            int16   
 4   creat_type_cd                     int8    
 5   adv_prim_id                       int16   
 6   dev_id                            int8    
 7   inter_type_cd                     int8    
 8   slot_id                           int8    
 9   spread_app_id                     int8    
 10  tags                              int8    
 11  app_first_class                   int8    
 12  app_second_class                  int8    
 13  age                               int8    
 14  city                              int16   
 15  city_rank                         int8    
 16  device_nam

In [11]:
# export mapping dict for datatypes
test_df.dtypes.to_csv("dtype_mapping.csv")

# export mapping dict for target encoding
with open('list_mapping_dict.json', 'w') as f:
    json.dump(list_mapping_dict, f)

In [12]:
with gzip.open('train_df.pkl', 'wb') as outputfile:
    pickle.dump(train_df, outputfile, protocol=-1)
    
with gzip.open('test_df.pkl', 'wb') as outputfile2:
    pickle.dump(test_df, outputfile2, protocol=-1)